Suppose we need to solve $\mathbf{A}\vec{x} = \vec{b}$, where $\mathbf{A}$ is m rows long and n columns wide. We consider only the case where $m=n$ (exactly determined). Ideally, $\vec{x} = \mathbf{A}^{-1} \vec{b}$, which involves calculating the determinant of $\mathbf{A}$ and then calculating the inverse (assuming $\mathbf{A}$ is non-singular). But that can quickly become complicated as the number of elements increases.

What all can we do with the matrix $\mathbf{A}$?

- We can swap the rows. This is just relabling the equations.
- We can swap the columns. This is just relabling the variables.
- We can form a linear combination of the equations without changing the system.

Warning : with a computer, precision can affect the singularity of a matrix!

# Gaussian Elimination
The steps to follow here are that:

- Subtract the first equation from the rest, appropriately scaled, such that the first variable is eliminated from all equations but the first.
- Now, subtract the second equation from the rest (*except first*), appropriately scaled, such that the second variable is eliminated from all equations but the second (and first).
- Keep doing this with the rest of the variables, keeping the overall process same.
- Finally, after all previous steps are finished, the last equation has all leading coefficients as zero except that for the last variable (essentially, an upper triangular matrix). Then, the value of this variable is back-substituted into the last but one equation to get the last but one variable's value. This way, all variables' values are obtained.

### Algorithm :

1. Identify the pivot $a_{11}$. Reduce elements of the first column (except $a_{11}$) to zero.
    - To do that, get multiplicative factors $l_{i1}$ for the ith row with respect to the 1st row $\Rightarrow l_{21} = \dfrac{a_{21}}{a_{11}} \cdots$
    - Do $a_{2j} \rightarrow a_{2j} - a_{1j} \cdot l_{21}, \hspace{0.5em} b_2 \rightarrow b_2 - b_1 \cdot l_{21}$. Repeat for all rows below.
    
2. Identify the new pivot $a_{22}$. Then,
    - Get $l_{i2} \hspace{0.5em}(i > 2)$. 
    - Do $a_{3j} \rightarrow a_{3j} - a_{2j} \cdot l_{2j}, \hspace{0.5em} b_2 \rightarrow b_2 - b_1 \cdot l_{21}$. Repeat for all rows below.

In [2]:
import numpy as np

In [3]:
def gaussian_eliminate(A, b):
    rows = np.shape(A)[0]
    columns = np.shape(A)[1]
    B = np.copy(A)
    c = np.copy(b)
    
    if(B[0][0] == 0):
        temp = B[0][:]
        B[0][:] = B[1][:]
        B[1][:] = temp
        temp = c[0]
        c[0] = c[1]
        c[1] = temp
        
    for i in range(rows):
        for j in range(i+1, rows):
            multfactor = B[j][i] / B[i][i]
            B[j][:] = B[j][:] - multfactor * B[i][:]
            c[j] = c[j] - multfactor * c[i]
            
    outputs = np.zeros(columns)
    
    for i in range(rows-1, -1, -1):
        summation = 0
        for j in range(columns-1, i, -1):
            summation += B[i][j] * outputs[j]
        answer = (c[i] - summation) / B[i][i]
        outputs[i] = answer
        
    return(outputs)

In [4]:
A = np.array([[3, 1, 5], [3, 3, 1], [4, 6, 4]], dtype=float)
b = np.array([[0., 1., 0.]], dtype=float).T
answer = gaussian_eliminate(A, b)
print(answer)


[ 0.65 -0.2  -0.35]


### LU Decomposition
When one has to solve a set of equations repeatedly, with only the coefficient vector changing, instead of Gaussian Elimination, LU Decomposition proves more efficient. Here, the original matrix is decomposed into the product of a lower triangular matrix L and an upper triangular matrix U. In the algorithm we perform, we set the diagonal elements of the lower triangular matrix to be unity. In this case:
- Set the non-unity, lower triangular matrix elements to be the respective multiplicative factors.
- The upper triangular matrix elements will be the upper triangular matrix you get at the penultimate step of Gaussian Elimination, before back substitution.
Now, we went from $\mathbf{A}\vec{x} = \vec{b}$ to $\mathbf{LU}\vec{x} = \vec{b}$. To solve for x, we write this as two equations : $$\mathbf{L}\vec{y} = \vec{b}; \quad \mathbf{U}\vec{x} = \vec{y}$$
Solving these two (both by back-substitution), we can get the solution for x.